In [2]:
import pandas as pd
import numpy as np

# --------------------------------
# 1. Load Input Data
# --------------------------------
input_file = "inventory_input.csv"
output_file = "procurement_priority_output.csv"

df = pd.read_csv(input_file, encoding="latin1")

# --------------------------------
# 2. Config Tables (Enterprise Style)
# --------------------------------
import yaml

with open("configs/service_policy.yaml", "r") as f:
    service_policy = yaml.safe_load(f)

with open("configs/priority_config.yaml", "r") as f:
    priority_config = yaml.safe_load(f)

with open("configs/lead_time_map.yaml", "r") as f:
    lead_time_map = yaml.safe_load(f)

df["Lead_Time_Months"] = df["Supplier_Region"].map(lead_time_map)

# --------------------------------
# 3. Lead Time Rules
# --------------------------------
df["Lead_Time_Months"] = df["Supplier_Region"].map(lead_time_map)

# --------------------------------
# 4. Service Level Policy
# --------------------------------
df["Z_Score"] = df["Category_Tier"].map(lambda x: service_policy.get(x, {"z": np.nan})["z"])

# --------------------------------
# 5. Forecast Error Assumption
# --------------------------------
df["Forecast_Error_Rate"] = 0.30  # replace with SKU-level MAPE if available

# --------------------------------
# 6. Core Inventory Calculations
# --------------------------------
df["Effective_Stock"] = df["Stock_On_Hand"] + df["On_Order"]
df["Lead_Time_Demand"] = df["Avg_Monthly_Demand"] * df["Lead_Time_Months"]

# --------------------------------
# 7. Safety Stock
# --------------------------------
df["Demand_Std_Dev"] = df["Avg_Monthly_Demand"] * df["Forecast_Error_Rate"]
df["Service_Level_Safety_Stock"] = df["Z_Score"] * df["Demand_Std_Dev"] * np.sqrt(df["Lead_Time_Months"])
df["Forecast_Error_Buffer"] = df["Avg_Monthly_Demand"] * df["Forecast_Error_Rate"] * df["Lead_Time_Months"]
df["Total_Safety_Stock"] = df["Service_Level_Safety_Stock"] + df["Forecast_Error_Buffer"]

# --------------------------------
# 8. Dynamic Min–Max Policy
# --------------------------------
df["Dynamic_Min"] = df["Lead_Time_Demand"] + df["Total_Safety_Stock"]
df["Dynamic_Max"] = df["Dynamic_Min"] + (df["Avg_Monthly_Demand"] * 2)

# --------------------------------
# 9. Stock-Out Risk
# --------------------------------
df["Projected_Stock_At_Arrival"] = df["Effective_Stock"] - df["Lead_Time_Demand"]
df["Stockout_Risk"] = np.select(
    [df["Projected_Stock_At_Arrival"] <= 0,
     df["Projected_Stock_At_Arrival"] <= df["Total_Safety_Stock"]],
    ["HIGH", "MEDIUM"],
    default="LOW"
)

# --------------------------------
# 10. Order Decision Logic
# --------------------------------
df["Order_Required"] = np.where(df["Effective_Stock"] < df["Dynamic_Min"], "YES", "NO")
df["Suggested_Order_Qty"] = np.where(df["Order_Required"] == "YES",
                                     df["Dynamic_Max"] - df["Effective_Stock"], 0).round(0)

# --------------------------------
# 11. Realistic Profit Calculation
# --------------------------------
# Convert unit cost and unit price to 12-month totals
df["COGS_12mo"] = df["Cost"] * df["Avg_Monthly_Demand"] * 12
df["Revenue_12mo"] = df["Revenue"] * df["Avg_Monthly_Demand"] * 12

# Fulfillable units = current stock + suggested order
df["Fulfillable_Units"] = np.maximum(df["Effective_Stock"] + df["Suggested_Order_Qty"], 0)

# Realistic profit = margin per unit × fulfillable units
df["Realistic_Profit"] = (df["Revenue"] - df["Cost"]) * df["Fulfillable_Units"]

# PL Tier thresholds
df["PL_Tier"] = np.select(
    [df["Realistic_Profit"] > 100000, df["Realistic_Profit"] > 0],
    ["HIGH_PROFIT", "LOW_PROFIT"],
    default="LOSS"
)

# --------------------------------
# 12. Weighted ABC Classification (Profit Contribution)
# --------------------------------
df["Profit_Share"] = df["Realistic_Profit"] / df["Realistic_Profit"].sum()
df = df.sort_values("Profit_Share", ascending=False)
df["Cumulative_Share"] = df["Profit_Share"].cumsum()

df["ABC_Class"] = np.select(
    [df["Cumulative_Share"] <= 0.80, df["Cumulative_Share"] <= 0.95],
    ["A", "B"],
    default="C"
)

# --------------------------------
# 13. XYZ Classification (Demand Variability)
# --------------------------------
df["CoV"] = df["Demand_Std_Dev"] / df["Avg_Monthly_Demand"].replace(0, np.nan)
df["XYZ_Class"] = np.select(
    [df["CoV"] < 0.25, df["CoV"] < 0.50],
    ["X", "Y"],
    default="Z"
)

# --------------------------------
# 14. Combined Matrix
# --------------------------------
df["ABC_XYZ"] = df["ABC_Class"].astype(str) + df["XYZ_Class"].astype(str)

# --------------------------------
# 15. Priority Scoring (Config-Driven with Breakdown)
# --------------------------------
def compute_priority_score(df, priority_config):
    abc = df["ABC_Class"].astype(str).map(priority_config["abc_weight"]).fillna(0).astype(float)
    xyz = df["XYZ_Class"].astype(str).map(priority_config["xyz_weight"]).fillna(0).astype(float)
    pl  = df["PL_Tier"].astype(str).map(priority_config["pl_weight"]).fillna(0).astype(float)

    tier = df["Category_Tier"].astype(str).map(priority_config["tier_weight"]).fillna(0).astype(float)
    risk = df["Stockout_Risk"].astype(str).map(priority_config["risk_weight"]).fillna(0).astype(float)
    order_flag = (df["Suggested_Order_Qty"] > 0).astype(int)

    score = (
        tier * priority_config["multipliers"]["tier"] +
        risk * priority_config["multipliers"]["risk"] +
        order_flag * priority_config["multipliers"]["order_flag"] +
        abc * priority_config["multipliers"]["abc"] +
        xyz * priority_config["multipliers"]["xyz"] +
        pl * priority_config["multipliers"]["pl"]
    )

    breakdown = (
        "Tier:" + tier.astype(str) +
        "|Risk:" + risk.astype(str) +
        "|Order:" + order_flag.astype(str) +
        "|ABC:" + abc.astype(str) +
        "|XYZ:" + xyz.astype(str) +
        "|PL:" + pl.astype(str)
    )

    return score, breakdown

df["Priority_Score"], df["Score_Breakdown"] = compute_priority_score(df, priority_config)

# --------------------------------
# 16. Final Output
# --------------------------------
final_cols = [
    "BarCode", "Description", "Category_Tier", "Supplier_Region",
    "Effective_Stock", "Lead_Time_Demand", "Total_Safety_Stock",
    "Dynamic_Min", "Dynamic_Max", "Projected_Stock_At_Arrival",
    "Stockout_Risk", "Suggested_Order_Qty", "ABC_Class", "XYZ_Class",
    "ABC_XYZ", "PL_Tier", "Realistic_Profit", "Priority_Score", "Score_Breakdown"
]

priority_df = df[final_cols].sort_values("Priority_Score", ascending=False)
priority_df.to_csv(output_file, index=False)

print(f"Procurement priority file generated: {output_file}")


Procurement priority file generated: procurement_priority_output.csv
